# Generate force coefficients

The following steps are performed to obtain all the artifacts for force coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Instantiate path manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CfPathManager
import pathlib

path_manager = CfPathManager(output_path=pathlib.Path("./output/pressure"))

Read parameters file

In [2]:
from cfdmod.use_cases.pressure.force.Cf_config import CfCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Cf_params.yaml")
post_proc_cfg = CfCaseConfig.from_file(config_path)

post_proc_cfg.force_coefficient

{'measurement_1': CfConfig(body='building2', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[0.0, 10.0, 20.0]), variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'mean', 'skewness', 'kurtosis', 'xtr_min', 'xtr_max', 'mean_qs'], transformation=TransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fixed_point=(0, 0, 0))),
 'measurement_2': CfConfig(body='marquise', sub_bodies=ZoningModel(x_intervals=[-inf, inf], y_intervals=[-inf, inf], z_intervals=[-inf, inf]), variables=['Cfx', 'Cfy', 'Cfz'], statistics=['min', 'max', 'std', 'mean', 'skewness', 'kurtosis', 'xtr_min', 'xtr_max', 'mean_qs'], transformation=TransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fixed_point=(0, 0, 0)))}

Read Mesh from file

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Calculate force coefficient

In [4]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.force.Cf_data import CfOutputs, process_Cf

cp_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")

for cfg_label, cfg in post_proc_cfg.force_coefficient.items():
    logger.info(f"Processing body {cfg.body} ...")

    cf_output: CfOutputs = process_Cf(
        mesh=mesh,
        body_cfg=post_proc_cfg.bodies[cfg.body],
        cfg=cfg,
        cp_path=cp_path,
        extreme_params=post_proc_cfg.extreme_values,
    )

    cf_output.save_outputs(body_label=cfg.body, cfg_label=cfg_label, path_manager=path_manager)

    logger.info(f"Processed body {cfg.body}!")

[2024-01-18 16:16:02,917] [INFO] - cfdmod - Processing body building2 ... (3330637806.py:7)
[2024-01-18 16:16:04,210] [INFO] - cfdmod - Processed body building2! (3330637806.py:19)
[2024-01-18 16:16:04,211] [INFO] - cfdmod - Processing body marquise ... (3330637806.py:7)
[2024-01-18 16:16:04,315] [INFO] - cfdmod - Processed body marquise! (3330637806.py:19)
